In [1]:
import tensorflow as tf
import numpy as np
import sys
import os
import functools
import time
import config
import shutil
from my_tools import define_scope
import my_tools

print("loaded libs")

loaded libs


In [2]:
print (len(os.listdir(config.PIC_SRC_DIR)))


params = {"result_dir": config.RESULT_DIR,
          "learning_rate": 1e-4,
          "img_size": config.CNN_IN_HEIGHT,
          "num_channels":config.CNN_IN_CH,
          "num_classes": config.CNN_OUTPUT_SIZE,
          "batch_size": config.BATCH_SIZE,
          "keep_probability": config.KEEP_PROB,
          "print_nth_step": config.PRINT_NTH}
print("loaded params")


def log_dir_name(learning_rate, num_dense_layers,
                 num_dense_nodes, activation):

    # The dir-name for the TensorBoard log-dir.
    s = "./19_logs/lr_{0:.0e}_layers_{1}_nodes_{2}_{3}/"

    # Insert all the hyper-parameters in the dir-name.
    log_dir = s.format(learning_rate,
                       num_dense_layers,
                       num_dense_nodes,
                       activation)

    return log_dir

100
loaded params


In [3]:
def parser(serialized_data):
    """
    Decode and map data properly
    """
    
    features = {"image" : tf.FixedLenFeature((),tf.string),
                   "class" : tf.FixedLenFeature((),tf.int64),
                   "x1" : tf.FixedLenFeature((),tf.float32),
                   "y1" : tf.FixedLenFeature((),tf.float32),
                   "x2" : tf.FixedLenFeature((),tf.float32),
                   "y2" : tf.FixedLenFeature((),tf.float32),
                   "index" : tf.FixedLenFeature((),tf.int64)}
    parsed_features = tf.parse_single_example(serialized_data, features)
    
    image_string = parsed_features["image"]
    image = tf.decode_raw(image_string, tf.uint8)
    image = tf.cast(image,dtype=tf.float32)
    
    
    label = tf.convert_to_tensor((tf.cast(parsed_features["class"], tf.float32),
                     parsed_features["x1"],
                     parsed_features["y1"],
                     parsed_features["x2"],
                     parsed_features["y2"]))
    index = tf.cast(parsed_features["index"], tf.int32)
    
    return image, label, index

In [4]:
#WRAPPERS FOR MODEL

class Model(object):
    """
    Model of NN
    """
    #Variable of model
    #accuracy = tf.reduce
    
    
### INITIALIZATION ####    
    def __init__(self, params):
        #self.permutation = np.random.permutation(range(len(os.listdir(config.PIC_SRC_DIR))))
        
        self.img_size = params["img_size"]
        self.num_channels = params["num_channels"]
        self.num_classes = params["num_classes"]
        self.lr = params["learning_rate"]
        self.keep_prob = params["keep_probability"]
        self.write_step = 10#params["print_nth_step"]
        self.is_training = True
        self.global_step = tf.Variable(0, False, dtype = tf.int32, name="global_step")
        #remove bellow
        #self.data = tf.placeholder(dtype= tf.float32, shape=[None,256,256,3])
        #self.target = tf.placeholder(dtype = tf.float32, shape=[None,5])
        #self.index = tf.placeholder(dtype= tf.int32,shape=[None,1])

        #FUNCTIONS
        self.data_pipeline
        self.prediction
        self.loss_op
        self.optimize
        self.summary

    

        
#FUNCTION DEFINITIONS
    @define_scope
    def data_pipeline(self):
        """
        loading of TFRecords
        """
        train_dataset = tf.data.TFRecordDataset(os.path.join(config.DATA_DIR,config.TFRECORD_NAMES[0]))
        test_dataset = tf.data.TFRecordDataset(os.path.join(config.DATA_DIR,config.TFRECORD_NAMES[1]))
        train_dataset =train_dataset.map(parser)
        test_dataset = test_dataset.map(parser)
        
        iterator = tf.data.Iterator.from_structure( train_dataset.output_types,
                                                    train_dataset.output_shapes)
        
        self.data, target, self.index = iterator.get_next()
        self.target = tf.reshape(target, shape = [5])
        self.data = tf.reshape(self.data,[-1, self.img_size, self.img_size, self.num_channels]) 
        
        self.train_init = iterator.make_initializer(train_dataset)
        #test_init = iterator.make_initializer(test_dataset)

        
   
 ##### NEURAL NETWORK #####   
    @define_scope
    def prediction(self):
        """
        Main body of neural network, takes data and labels as input,
        returns feature map of photo
        """
    
        #1 conv layer
        conv1 = tf.layers.conv2d(inputs = self.data, 
                             filters = 32,
                             kernel_size = [5,5],
                             strides = 1,
                             padding = "same",
                             activation = tf.nn.relu,
                             name = 'conv1')
        
        #1 pool layer, img size reduced by 1/4
        pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                        pool_size = 2, 
                                        strides = 2,
                                        padding = "same",
                                        name = 'pool1')

        #2 conv layer
        conv2 = tf.layers.conv2d(inputs = pool1, 
                             filters = 64,
                             kernel_size = 5,
                             strides = 1,
                             padding = "same",
                             activation = tf.nn.relu,
                             name = 'conv2')

        #2 pool overal image size reduced totaly by factor of 1/16
        pool2 = tf.layers.max_pooling2d(inputs = conv2,
                                        pool_size = 2, 
                                        strides = 2,
                                        padding = "same",
                                        name = 'pool2')
        
        
        pool2_flat = tf.reshape(pool2,[-1, 64*64*64])

        
        dense = tf.layers.dense(inputs = pool2_flat,
                            units = 512,
                            activation = tf.nn.relu,
                            name = 'fc')
        
        dropout = tf.layers.dropout(dense,
                                    rate = self.keep_prob,
                                    training = self.is_training,
                                    name = 'dropout')
        
        dense2 = tf.layers.dense(inputs = dropout,
                             units = self.num_classes,
                             activation = tf.nn.relu,
                             name = "logits")
        
        return tf.nn.softmax(tf.reshape(dense2, shape=[5]))
    
#### LOSS ####    
    @define_scope
    def loss_op(self):
        """
        loss
        """
        self.loss = tf.losses.mean_squared_error(labels = self.target, predictions = self.prediction)
    
        return self.loss
    
##### OPTIMIZER #####    
    @define_scope    
    def optimize(self):
        """
        Optimizer of model
        """
        self.opt = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.loss, global_step=self.global_step)
        a = tf.cast(1,tf.int32) #trash line so fetcher doesnt return errors
        return a
       
    
    
    
###summary####    
    @define_scope
    def summary(self):
        '''
        Create summaries to write on TensorBoard
        '''
        with tf.name_scope('summaries'):
            tf.summary.scalar('loss', self.loss)
            tf.summary.histogram('histogram_loss', self.loss)
            self.summary_op = tf.summary.merge_all()
            
            
            
#### TRAIN ###        
    def train_one_time(self, sess, ckpt_dir, writer, saver, step, epoch):
        """
        Training op for model
        """
        start_time = time.time()
        sess.run(self.train_init)
        self.is_training = True
        total_loss = 0
        num_batches = 0
        self.training = True
        try :
            while True:
                #train
                l, _,summary = sess.run([self.loss_op, self.optimize, self.summary_op])
                writer.add_summary(summary, global_step=step)
                if (step) % self.write_step == 0:
                    print('Loss at step {0}: {1}'.format(step, l))
                total_loss += l
                num_batches += 1
                step+=1
                self.global_step = tf.convert_to_tensor(step)
        except tf.errors.OutOfRangeError:
            pass
        except KeyboardInterrupt:
            print("keyboard interrupt")
            pass
        saver.save(sess, ckpt_dir, global_step = step)
        print('Average loss at epoch {0}: {1}'.format(epoch, total_loss/num_batches))
        print('Took: {0} seconds'.format(time.time() - start_time))
        return step
    
    
 #### EVALUATE ####    
    def evaluate(self, step, epoch):
        """
        Evaluate once from test
        """
        start_time = time.time()
        sess.run(self.val_init())
        self.is_training = False
        try:
            l, summary = sess.run([self.loss_op, self.summary_op])
            writer.add_summary(summary, global_step=step)
        except tf.errors.OutOfRangeError:
            pass
        print('Loss at validaiton epoch {0}: {1}'.format(epoch, l))
        print('Took: {0} seconds'.format(time.time() - start_time))
        
        
        
 ### TO DO ####   
    def train_n_times(self, n_times):
        pass
        
    

In [5]:
total_epoch = 2
def main(remove_results = False):
    result_dir = config.RESULT_DIR
    
    if remove_results == True:
        shutil.rmtree(result_dir, ignore_errors=True)
    #Check for dirs, if not present make them
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    ckpt_dir=os.path.join(result_dir,"ckpt")
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
    
    graph = tf.Graph()
    #Get name as default graph
    with graph.as_default():

        print("Starting Session")
        #Assign name to session, assign it's default graph as graph
        with tf.Session(graph=graph) as sess:
            
            #Creating summary writer 
            writer = tf.summary.FileWriter(ckpt_dir, graph=graph)
                
            #Initialization of Model, load all Model functions returning variables
            model = Model(params)
            
            #Assign Initializer
            init = tf.global_variables_initializer()
            
            #Creating save for model session for future saving and restoring model
            saver = tf.train.Saver()
            
            #sess.run(init)
            #Loading last checkpoint
            ckpt = tf.train.get_checkpoint_state(result_dir)
            if ckpt and ckpt.model_checkpoint_path:
                #if ckpt found load it and load global step
                saver.restore(sess, ckpt.model_checkpoint_path)
                print("Found checkpoint")
                step = int(os.path.basename(ckpt.model_checkpoint_path).split('-')[1])
            else: 
                sess.run(init)
                step = 0
             
           
            print("Current step: {0}".format(step))        
            #Add graph
            writer.add_graph(graph)
            #Training
            print("Starting Training")
            for epoch in range(1,total_epoch):
                try: 
                    step = model.train(sess, ckpt_dir, writer, saver, step, epoch)
                except KeyboardInterrupt:
                    print("keyboard interrupt")
                    pass
            print("Closing session and saving results")
            print(model.global_step.eval(), step)
            saver.save(sess, ckpt_dir, global_step = step)
        
        #Merge all summaries
        writer.flush()
        #writer.add_graph(graph)
        writer.close()
        print("Closed summary, work finnished")

In [6]:
### tf.reset_default_graph()
if __name__ == '__main__':
    check = True
    main(remove_results=check)
    check = False
    main(remove_results=check)

Starting Session
Initialized Model.data_pipeline
Initialized Model.prediction
Initialized Model.loss_op
Initialized Model.optimize
Initialized Model.summary
Current step: 0
Starting Training
Loss at step 0: 0.252532958984
Loss at step 10: 0.215399175882
Loss at step 20: 0.386651605368
Loss at step 30: 0.466766357422
Loss at step 40: 0.317413300276
Loss at step 50: 0.402007550001
Average loss at epoch 1: 0.439363815884
Took: 13.6275708675 seconds
Closing session and saving results
(60, 60)
Closed summary, work finnished
Starting Session
Initialized Model.data_pipeline
Initialized Model.prediction
Initialized Model.loss_op
Initialized Model.optimize
Initialized Model.summary
INFO:tensorflow:Restoring parameters from results/ckpt-60
Found checkpoint
Current step: 60
Starting Training
Loss at step 60: 0.444720476866
Loss at step 70: 0.412274181843
keyboard interrupt
Average loss at epoch 1: 0.463223468926
Took: 9.27096700668 seconds
Closing session and saving results
(78, 78)
Closed summar

In [7]:
"""
tensorboard --logdir="results/"
"""

'\ntensorboard --logdir="results/"\n'

In [9]:
check = False
main(remove_results=check)
main(remove_results=check)

Starting Session
Initialized Model.data_pipeline
Initialized Model.prediction
Initialized Model.loss_op
Initialized Model.optimize
Initialized Model.summary
INFO:tensorflow:Restoring parameters from results/ckpt-198
Found checkpoint
Current step: 198
Starting Training
Loss at step 200: 0.441424578428
Loss at step 210: 0.287077844143
Loss at step 220: 0.589593529701
Loss at step 230: 0.322869867086
Loss at step 240: 0.576019287109
Loss at step 250: 0.499688714743
Average loss at epoch 1: 0.439649326354
Took: 15.8380219936 seconds
Closing session and saving results
(258, 258)
Closed summary, work finnished
Starting Session
Initialized Model.data_pipeline
Initialized Model.prediction
Initialized Model.loss_op
Initialized Model.optimize
Initialized Model.summary
INFO:tensorflow:Restoring parameters from results/ckpt-258
Found checkpoint
Current step: 258
Starting Training
Loss at step 260: 0.596112072468
Loss at step 270: 0.51167601347
Loss at step 280: 0.589593529701
Loss at step 290: 0.5